#### Siyuan "Kevin" Peng, Yuanzhe "Siris" Zheng, Yanlin "Jacky" Liu
![Image of Yaktocat](https://cdn.mos.cms.futurecdn.net/rLh7Dh7EKo8F6zmDtXYp8W.jpg)
# Table of Contents
1. [Introduction](#introduction)
2. [Install packages](#install-pkg)
3. [Data Dowloading](#data-download)
4. [Preprocessing](#preprocessing)<br>
    a. [Load and Clean Dataset](#load-and-clean)<br>
    b. [Data Analysis and Visualization](#data-ana-vis)
5. [Machine Learning Model](#ml-model)<br>
    a. [What and Why](#what-why)<br>
    b. [Training](#training)<br>
    c. [Result Anlysis and Demonstration](#result-ana-demon)
6. [Future Application](#future-app)
7. [Reference and External Link](#ref-and-extlink)

## 1. Introduction <a name="introduction"></a>
TODO:

## 2. Install Packages <a name="install-pkg"></a>
```
pip install kaggle numpy matplotlib pandas
```
or use environment.yml to install packages in conda environment
```
conda env update -f environment.yml
```
## 3. Data Downloading <a name="data-download"></a>

In [1]:
import kaggle
# remember to put kaggle.json to your C:/username/.kaggle
!kaggle datasets download -d ashaheedq/video-games-sales-2019

video-games-sales-2019.zip: Skipping, found more recently modified local copy (use --force to force download)


or directly download from kaggle webpage: [https://www.kaggle.com/ashaheedq/video-games-sales-2019](https://www.kaggle.com/ashaheedq/video-games-sales-2019)_
## 4. Preprocessing <a name="preprocessing"></a>
### Load and Clean Data <a name="load-and-clean"></a>

In [120]:
#TODO:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
import locale

locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
df = pd.read_csv("vgsales-12-4-2019.csv")
additional = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-07-30/video_games.csv")
additional.head()

,number,game,release_date,price,owners,developer,publisher,average_playtime,median_playtime,metascore
0,1,Half-Life 2,"Nov 16, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,110.0,66.0,96.0
1,3,Counter-Strike: Source,"Nov 1, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,236.0,128.0,88.0
2,21,Counter-Strike: Condition Zero,"Mar 1, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,10.0,3.0,65.0
3,47,Half-Life 2: Deathmatch,"Nov 1, 2004",4.99,"5,000,000 .. 10,000,000",Valve,Valve,0.0,0.0,NaN
4,36,Half-Life: Source,"Jun 1, 2004",9.99,"2,000,000 .. 5,000,000",Valve,Valve,0.0,0.0,NaN


In [121]:
additional = additional.dropna(subset = ['owners'])
additional = additional.reset_index(drop = True)
additional['Critic_Score'] = additional['metascore']/10
for i in range(len(additional)):
    str(additional.loc[i, 'owners'])
    nums = additional.loc[i, 'owners'].split('\xa0..\xa0')
#     print(nums)
    additional.loc[i, 'owners'] = float((locale.atoi(nums[1]) - locale.atoi(nums[0])) / 2000000)
#     print(additional.loc[i, 'owners'])

In [122]:
additional = additional.drop(columns = ['number', 'price', 'average_playtime', 'median_playtime'])
additional.head()

,game,release_date,owners,developer,publisher,metascore,Critic_Score
0,Half-Life 2,"Nov 16, 2004",5,Valve,Valve,96.0,9.6
1,Counter-Strike: Source,"Nov 1, 2004",5,Valve,Valve,88.0,8.8
2,Counter-Strike: Condition Zero,"Mar 1, 2004",5,Valve,Valve,65.0,6.5
3,Half-Life 2: Deathmatch,"Nov 1, 2004",2.5,Valve,Valve,NaN,NaN
4,Half-Life: Source,"Jun 1, 2004",1.5,Valve,Valve,NaN,NaN


In [123]:
additional['Name'] = additional['game']
additional['Developer'] = additional['developer']
additional['Global_Sales'] = additional['owners']
additional = additional.drop(columns=['metascore', 'release_date', 'publisher', 'game', 'developer', 'owners'])
df = df.drop(columns=['Rank', 'basename', 'Total_Shipped', 'Platform', 'Publisher', 'VGChartz_Score', 
                      'Last_Update', 'url', 'status', 'Vgchartzscore', 'img_url', 'ESRB_Rating', 'Year'])
pd.merge(df, additional, on = ['Name', 'Developer'] , how = 'outer')
df.head()

,Name,Genre,Developer,Critic_Score,User_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales
0,Wii Sports,Sports,Nintendo EAD,7.7,NaN,NaN,NaN,NaN,NaN,NaN
1,Super Mario Bros.,Platform,Nintendo EAD,10.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Racing,Nintendo EAD,8.2,9.1,NaN,NaN,NaN,NaN,NaN
3,PlayerUnknown's Battlegrounds,Shooter,PUBG Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wii Sports Resort,Sports,Nintendo EAD,8.0,8.8,NaN,NaN,NaN,NaN,NaN


In [124]:
df = df.dropna(subset=['Developer', 'Genre'])
df = df.reset_index(drop = True)
for i in range(len(df)):
    df.loc[i, 'Developer'] = str(df.loc[i, 'Developer'])
le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown = 'ignore')
df['Developer'] = le.fit_transform(df['Developer'])
df['Genre'] = le.fit_transform(df['Genre'])
df_temp = pd.DataFrame(ohe.fit_transform(df[['Genre']]).toarray())

In [125]:
df = df.join(df_temp)
df = df.drop(columns = ['Genre', 'User_Score'])
df = df.dropna(subset=['Global_Sales'])
df = df.reset_index(drop = True)
df = df[df['Global_Sales'] != 0.0]

In [126]:
df

,Name,Developer,Critic_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,0,1,...,10,11,12,13,14,15,16,17,18,19
0,Grand Theft Auto V,5783,9.4,20.32,6.37,9.85,0.99,3.12,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Grand Theft Auto V,5783,9.7,19.39,6.06,9.71,0.60,3.02,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Grand Theft Auto: Vice City,5783,9.6,16.15,8.41,5.49,0.47,1.78,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Grand Theft Auto V,5783,NaN,15.86,9.06,5.33,0.06,1.42,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Call of Duty: Black Ops 3,7108,NaN,15.09,6.18,6.05,0.41,2.44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18028,FirePower for Microsoft Combat Flight Simulator 3,6118,NaN,0.01,NaN,0.00,NaN,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
18029,Tom Clancy's Splinter Cell,7205,NaN,0.01,NaN,0.00,NaN,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
18030,Ashita no Joe 2: The Anime Super Remix,1225,NaN,0.01,NaN,NaN,0.01,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18031,Tokyo Yamanote Boys for V: Main Disc,5684,NaN,0.01,NaN,NaN,0.01,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.shape

(12962, 30)

### Data Analysis and Visualization' <a name="data-ana-vis"></a>

In [6]:
#TODO:

## 5. Machine Learning Model <a name="ml-model"></a>
In this section, we are going to implement several models and predict global sales. In the world of machine learning, people
can split datas into two groups: numerical data and categorical data. Numerical data is everything that represented by numbers (integer
and floating point). It's continuous. Categorical data, however, is discrete. Different models will be used to predict these two type of data.

It is obvious to predict sales as numerical data but we have the accuracy concern(we will see accuracy in the **Result Analysis and Demonstration** section)
since the data may not demonstrate a strong linear trend. Therefore, we hope to predict it as categorical data: sale score is divided into 4 categories.
Games in ">10" category are expected to sell so greate that its name will left in history -- Grand Theft Auto, Pokemon, Call of duty and etc. You name it.
Games in "5-10" category are sold less than the top ones, but they are still great games. "5-1" games are good games. there are still large amount of customer want to
put them into their gaming library. The rest of games can be put into "1-0" categories. We respect the efforts that game developers put into them but they are relatively
niche.
### What and Why <a name="what-why"></a>

We want to use *multiple linear regression* for predicting numerical sale number. The reason is that we intend to investigate
how strong the relationship is between many independent variables (in this case, critic score, developers and other variables) and
one dependent variable -- sale score. We made several assumptions for using multiple linear regression.
 - Homogeneity of Variance: the size of the error in our prediction doesn't change a lot
 - Independence of Observations: each game is independent of others.
 - Linearity: the line of best fit through the data point is a straight line.

However, one noticeable issue is that there is no way we can check the linearity due to the fact that we have
multiple categories. Therefore, we also hope to perform *multiple nonlinear regression*, i.e. we will fit a curve
instead of a line for the data.

Several models will be used for the prediction of categorical sale number: *Random forest*, *k-nearest neighbors* (KNN) and
*Support vector machine*(SVM)

Single decision tree suffers from a high variance, which makes them less accurate than other models. However, random forest fixes
this problem. Benefits of using random forests:
 -  bagging and bootstrap reduce the output variance
 -  Able to handle large dataset with high dimensionality (which is our datset)

k-nearest neightbors, as one of the most famous classifications algorithm, surely have many positive sides:
 - No training period
 - Easily to add new data
 - easy to implement

Here is the advantages of choosing support vector machine as one of our algorithem.
 -  Effective in high dimensional spaces
 -  Use a subset of training set in the decision function and, therefore, prevent overfitting
 -  memory efficient

### Training <a name="training"></a>

In [8]:
#TODO:

### Result Anlysis and Demonstration <a name="result-ana-demon"></a>
TODO:
## 6. Future Application <a name="future-app"></a>
TODO:
## 7. Reference and External Link <a name="ref-and-extlink"></a>
#### Want to to know more about multiple linear regression?
 - https://www.scribbr.com/statistics/multiple-linear-regression/
 - https://en.wikipedia.org/wiki/Linear_regression
 - https://towardsdatascience.com/understanding-multiple-regression-249b16bde83e
#### Extend materials for support vector machine
 - https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47
 - https://www.youtube.com/watch?v=1NxnPkZM9bc